In [1]:
import pandas as pd 
import numpy as np 
from datetime import datetime, timedelta

In [2]:
data2=pd.read_excel("sampletesting.xlsx",sheet_name="Sheet1")
data1=pd.read_excel("sampletesting.xlsx",sheet_name="Sheet2")

In [5]:
print(data2.describe)

<bound method NDFrame.describe of           Date                    Customer       Village  Total_L
0   2025-06-01                   Gopalbhai        Shilly     35.0
1   2025-06-01            Ramprasad Khatik     Rajasthan    400.0
2   2025-06-10                  Vikramsinh     Mithapura     30.0
3   2025-06-11         Prahladbhai -Mantry  Bhalod Dairy      7.0
4   2025-06-12               V S Stud Farm      Waghodia    400.0
..         ...                         ...           ...      ...
166 2025-07-30         Hemendrabhai Parmar    Panchdevla     50.0
167 2025-07-30                  Sundarbhai        Siyali     13.0
168 2025-07-31  Kamleshbhai Vasava -Mantry         Moran      1.0
169 2025-07-31           Kiranbhai -Mantry      Talodara      1.0
170 2025-07-31                   Kiritbhai      Sindhrot     30.0

[171 rows x 4 columns]>


In [8]:
def analyze_sales_data(data1, data2):
    """
    Analyze sales data to identify targets for mantri communication and village focus
    """
    # Convert date column if needed
    data1['Date'] = pd.to_datetime(data1['Date'])
    data2['Date'] = pd.to_datetime(data2['Date'])
    
    # Calculate key metrics from Data1 (village level)
    data1['Conversion_Rate'] = (data1['Contact_In_Group'] / data1['Sabhasad'] * 100).round(2)
    data1['Conversion_Rate'] = data1['Conversion_Rate'].replace([np.inf, -np.inf], 0).fillna(0)
    data1['Untapped_Potential'] = data1['Sabhasad'] - data1['Contact_In_Group']
    data1['Sales_Per_Contact'] = (data1['Total_L'] / data1['Contact_In_Group']).round(2)
    data1['Sales_Per_Contact'] = data1['Sales_Per_Contact'].replace([np.inf, -np.inf], 0).fillna(0)
    
    # Calculate priority score for villages
    data1['Priority_Score'] = (
        (data1['Untapped_Potential'] / data1['Untapped_Potential'].max() * 50) +
        ((100 - data1['Conversion_Rate']) / 100 * 50)
    ).round(2)
    
    # Analyze recent sales from Data2 (customer level)
    # Since we don't have customer contact info, we'll analyze at village level
    recent_sales = data2.groupby('Village').agg({
        'Total_L': ['sum', 'count'],
        'Date': 'max'
    }).reset_index()
    
    # Flatten the column names
    recent_sales.columns = ['Village', 'Recent_Sales_L', 'Recent_Customers', 'Last_Sale_Date']
    
    # Calculate days since last sale
    recent_sales['Days_Since_Last_Sale'] = (datetime.now() - recent_sales['Last_Sale_Date']).dt.days
    
    # Merge with Data1
    analysis_df = data1.merge(recent_sales, on='Village', how='left')
    analysis_df['Recent_Sales_L'] = analysis_df['Recent_Sales_L'].fillna(0)
    analysis_df['Recent_Customers'] = analysis_df['Recent_Customers'].fillna(0)
    analysis_df['Days_Since_Last_Sale'] = analysis_df['Days_Since_Last_Sale'].fillna(999)
    
    # Generate recommendations for mantris
    recommendations = []
    
    for _, row in analysis_df.iterrows():
        village = row['Village']
        mantri = row['Mantri_Name']
        mobile = row['Mantri_Mobile']
        taluka = row['Taluka']
        district = row['District']
        
        # Recommendation logic
        if row['Conversion_Rate'] < 20:
            recommendations.append({
                'Village': village,
                'Taluka': taluka,
                'District': district,
                'Mantri': mantri,
                'Mobile': mobile,
                'Action': 'Send Marketing Team',
                'Reason': f'Low conversion rate ({row["Conversion_Rate"]:.1f}%) - Only {row["Contact_In_Group"]} of {row["Sabhasad"]} sabhasad contacted',
                'Priority': 'High',
                'Score': row['Priority_Score']
            })
        elif row['Untapped_Potential'] > 30:
            recommendations.append({
                'Village': village,
                'Taluka': taluka,
                'District': district,
                'Mantri': mantri,
                'Mobile': mobile,
                'Action': 'Call Mantri for Follow-up',
                'Reason': f'High untapped potential ({row["Untapped_Potential"]} sabhasad not contacted)',
                'Priority': 'High',
                'Score': row['Priority_Score']
            })
        elif row['Days_Since_Last_Sale'] > 30:
            recommendations.append({
                'Village': village,
                'Taluka': taluka,
                'District': district,
                'Mantri': mantri,
                'Mobile': mobile,
                'Action': 'Check on Mantri',
                'Reason': f'No recent sales ({row["Days_Since_Last_Sale"]} days since last sale)',
                'Priority': 'Medium',
                'Score': row['Priority_Score']
            })
        elif row['Sales_Per_Contact'] > 10:
            recommendations.append({
                'Village': village,
                'Taluka': taluka,
                'District': district,
                'Mantri': mantri,
                'Mobile': mobile,
                'Action': 'Provide More Stock',
                'Reason': f'High sales per contact ({row["Sales_Per_Contact"]}L per contact)',
                'Priority': 'Medium',
                'Score': row['Priority_Score']
            })
        else:
            recommendations.append({
                'Village': village,
                'Taluka': taluka,
                'District': district,
                'Mantri': mantri,
                'Mobile': mobile,
                'Action': 'Regular Follow-up',
                'Reason': 'Steady performance - maintain relationship',
                'Priority': 'Low',
                'Score': row['Priority_Score']
            })
    
    return pd.DataFrame(recommendations), analysis_df

def generate_mantri_messages(recommendations):
    """
    Generate personalized WhatsApp messages for mantris based on recommendations
    """
    messages = []
    
    for _, row in recommendations.iterrows():
        if row['Action'] == 'Send Marketing Team':
            message = f"""
Namaste {row['Mantri']} Ji!

Aapke kshetra {row['Village']} mein humare calcium supplement ki conversion rate kam hai. 
Humari marketing team aapke yaha demo dene aayegi. 
Kripya taiyaari rakhein aur sabhi dudh utpadakon ko soochit karein.

Dhanyavaad,
Calcium Supplement Team
"""
        elif row['Action'] == 'Call Mantri for Follow-up':
            message = f"""
Namaste {row['Mantri']} Ji!

Aapke kshetra {row['Village']} mein bahut se aise farmers hain jo abhi tak humare product se anabhijit hain. 
Kripya unse sampark karein aur unhe product ke fayde batayein. 
Aapke liye special commission offer hai agle 10 customers ke liye.

Dhanyavaad,
Calcium Supplement Team
"""
        elif row['Action'] == 'Check on Mantri':
            message = f"""
Namaste {row['Mantri']} Ji!

Humne dekha ki aapke kshetra {row['Village']} mein kuch samay se sales nahi hue hain.
Kya koi samasya hai? Kya hum aapki kisi tarah madad kar sakte hain?

Kripya hame batayein.

Dhanyavaad,
Calcium Supplement Team
"""
        elif row['Action'] == 'Provide More Stock':
            message = f"""
Namaste {row['Mantri']} Ji!

Badhai ho! Aapke kshetra {row['Village']} mein humare product ki demand badh rahi hai.
Kya aapko aur stock ki zaroorat hai? Hum jald se jald aapko extra stock bhej denge.

Dhanyavaad,
Calcium Supplement Team
"""
        else:
            message = f"""
Namaste {row['Mantri']} Ji!

Aapke kshetra {row['Village']} mein humare product ki sales theek chal rahi hain.
Kripya aise hi continue rakhein aur koi bhi sujhav ho toh hame batayein.

Dhanyavaad,
Calcium Supplement Team
"""
        
        messages.append({
            'Mantri': row['Mantri'],
            'Mobile': row['Mobile'],
            'Village': row['Village'],
            'Action': row['Action'],
            'Message': message,
            'Priority': row['Priority']
        })
    
    return pd.DataFrame(messages)

def identify_demo_locations(analysis_df, top_n=5):
    """
    Identify the best locations for demos based on various factors
    """
    # Calculate a demo score based on multiple factors
    analysis_df['Demo_Score'] = (
        (analysis_df['Untapped_Potential'] / analysis_df['Untapped_Potential'].max() * 40) +
        ((100 - analysis_df['Conversion_Rate']) / 100 * 30) +
        (analysis_df['Recent_Sales_L'] / analysis_df['Recent_Sales_L'].max() * 30)
    ).round(2)
    
    # Get top locations for demos
    demo_locations = analysis_df.nlargest(top_n, 'Demo_Score')[
        ['Village', 'Taluka', 'District', 'Mantri_Name', 'Mantri_Mobile', 
         'Conversion_Rate', 'Untapped_Potential', 'Demo_Score']
    ]
    
    return demo_locations

# Example usage with sample data structure
def main():
    data2=pd.read_excel("sampletesting.xlsx",sheet_name="Sheet1")
    data1=pd.read_excel("sampletesting.xlsx",sheet_name="Sheet2")
    # Generate recommendations
    recommendations, analysis = analyze_sales_data(data1, data2)
    
    print("RECOMMENDED ACTIONS:")
    print(recommendations.sort_values('Score', ascending=False).to_string(index=False))
    
    # Generate messages for mantris
    mantri_messages = generate_mantri_messages(recommendations)
    
    print("\nMANTRI MESSAGES:")
    for _, msg in mantri_messages.iterrows():
        print(f"\nTo: {msg['Mantri']} ({msg['Mobile']}) - {msg['Village']}")
        print(f"Action: {msg['Action']}")
        print(f"Message: {msg['Message']}")
    
    # Identify demo locations
    demo_locations = identify_demo_locations(analysis)
    
    print("\nTOP DEMO LOCATIONS:")
    print(demo_locations.to_string(index=False))
    
    return recommendations, mantri_messages, demo_locations

if __name__ == "__main__":
    recommendations, mantri_messages, demo_locations = main()

RECOMMENDED ACTIONS:
             Village    Taluka District                      Mantri       Mobile                    Action                                                              Reason Priority   Score
   BORSAD MAIN DAIRY    BORSAD    ANAND BHARATBHAI PARMAR(CHAIRMAN)   9879726187 Call Mantri for Follow-up             High untapped potential (2677.0 sabhasad not contacted)     High   88.24
             SAIJPUR    BORSAD    ANAND                    RAJUBHAI   8511880200       Send Marketing Team   Low conversion rate (3.8%) - Only 71.0 of 1885 sabhasad contacted     High   82.00
            BAJIPURA  KHAMBHAT    ANAND   MANUBHAI SOMABHAI CHAUHAN   9824643396       Send Marketing Team   Low conversion rate (2.9%) - Only 31.0 of 1062 sabhasad contacted     High   67.80
           HATHIPURA   ANKALAV    ANAND           JESANGBHAI CHAVDA   9427828741       Send Marketing Team    Low conversion rate (1.4%) - Only 10.0 of 740 sabhasad contacted     High   62.96
              SANDH

## ML based suggestions

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

def enhanced_analyze_sales_data(data1, data2):
    """
    Enhanced analysis with ML components for better predictions
    """
    
    data1['Date'] = pd.to_datetime(data1['Date'])
    data2['Date'] = pd.to_datetime(data2['Date'])
    
    # Calculate basic metrics
    data1['Conversion_Rate'] = (data1['Contact_In_Group'] / data1['Sabhasad'] * 100).round(2)
    data1['Conversion_Rate'] = data1['Conversion_Rate'].replace([np.inf, -np.inf], 0).fillna(0)
    data1['Untapped_Potential'] = data1['Sabhasad'] - data1['Contact_In_Group']
    data1['Sales_Per_Contact'] = (data1['Total_L'] / data1['Contact_In_Group']).round(2)
    data1['Sales_Per_Contact'] = data1['Sales_Per_Contact'].replace([np.inf, -np.inf], 0).fillna(0)
    
    # Analyze recent sales
    recent_sales = data2.groupby('Village').agg({
        'Total_L': ['sum', 'count'],
        'Date': 'max'
    }).reset_index()
    
    recent_sales.columns = ['Village', 'Recent_Sales_L', 'Recent_Customers', 'Last_Sale_Date']
    recent_sales['Days_Since_Last_Sale'] = (datetime.now() - recent_sales['Last_Sale_Date']).dt.days
    
    # Merge data
    analysis_df = data1.merge(recent_sales, on='Village', how='left')
    analysis_df['Recent_Sales_L'] = analysis_df['Recent_Sales_L'].fillna(0)
    analysis_df['Recent_Customers'] = analysis_df['Recent_Customers'].fillna(0)
    analysis_df['Days_Since_Last_Sale'] = analysis_df['Days_Since_Last_Sale'].fillna(999)
    
    # ML Component 1: Village Clustering for Segmentation
    analysis_df = apply_village_clustering(analysis_df)
    
    # ML Component 2: Predict Sales Potential
    analysis_df = predict_sales_potential(analysis_df)
    
    # ML Component 3: Action Recommendation Classifier
    analysis_df = predict_recommended_actions(analysis_df)
    
    # Generate recommendations based on ML predictions
    recommendations = generate_ml_recommendations(analysis_df)
    
    return recommendations, analysis_df

def apply_village_clustering(analysis_df):
    """
    Use K-Means clustering to segment villages into groups
    """
    # Prepare features for clustering
    cluster_features = analysis_df[[
        'Conversion_Rate', 'Untapped_Potential', 'Sales_Per_Contact', 
        'Recent_Sales_L', 'Days_Since_Last_Sale'
    ]].fillna(0)
    
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(cluster_features)
    
    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(scaled_features)
    
    # Add clusters to dataframe
    analysis_df['Cluster'] = clusters
    
    # Name the clusters based on characteristics
    cluster_names = {
        0: 'High Potential - Low Engagement',
        1: 'Steady Performers', 
        2: 'Underperforming',
        3: 'New/Developing'
    }
    
    analysis_df['Segment'] = analysis_df['Cluster'].map(cluster_names)
    
    return analysis_df

def predict_sales_potential(analysis_df):
    """
    Predict sales potential for each village using Random Forest
    """
    # Prepare features for prediction
    prediction_features = analysis_df[[
        'Sabhasad', 'Contact_In_Group', 'Conversion_Rate', 
        'Untapped_Potential', 'Recent_Sales_L', 'Days_Since_Last_Sale'
    ]].fillna(0)
    
    # Target variable: Total_L (current sales)
    target = analysis_df['Total_L'].fillna(0)
    
    # Only train if we have enough data
    if len(prediction_features) > 10:
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            prediction_features, target, test_size=0.2, random_state=42
        )
        
        # Train model
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        
        # Make predictions
        predictions = model.predict(prediction_features)
        
        # Calculate feature importance
        feature_importance = pd.DataFrame({
            'feature': prediction_features.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        # Add predictions to dataframe
        analysis_df['Predicted_Sales'] = predictions
        analysis_df['Sales_Gap'] = analysis_df['Predicted_Sales'] - analysis_df['Total_L']
    else:
        # Fallback if not enough data
        analysis_df['Predicted_Sales'] = analysis_df['Total_L']
        analysis_df['Sales_Gap'] = 0
    
    return analysis_df

def predict_recommended_actions(analysis_df):
    """
    Use ML to predict the best action for each village
    """
    # Define actions based on rules (for training data)
    analysis_df['Action_Label'] = np.where(
        analysis_df['Conversion_Rate'] < 20, 'Send Marketing Team',
        np.where(
            analysis_df['Untapped_Potential'] > 30, 'Call Mantri for Follow-up',
            np.where(
                analysis_df['Days_Since_Last_Sale'] > 30, 'Check on Mantri',
                np.where(
                    analysis_df['Sales_Per_Contact'] > 10, 'Provide More Stock',
                    'Regular Follow-up'
                )
            )
        )
    )
    
    # Prepare features for classification
    classification_features = analysis_df[[
        'Conversion_Rate', 'Untapped_Potential', 'Sales_Per_Contact',
        'Recent_Sales_L', 'Days_Since_Last_Sale', 'Sales_Gap'
    ]].fillna(0)
    
    # Target variable: Action_Label
    target = analysis_df['Action_Label']
    
    # Only train if we have enough data
    if len(classification_features) > 10 and len(target.unique()) > 1:
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            classification_features, target, test_size=0.2, random_state=42, stratify=target
        )
        
        # Train classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)
        
        # Make predictions
        predictions = clf.predict(classification_features)
        prediction_proba = clf.predict_proba(classification_features)
        
        # Add predictions to dataframe
        analysis_df['ML_Recommended_Action'] = predictions
        analysis_df['Action_Confidence'] = np.max(prediction_proba, axis=1)
    else:
        # Fallback to rule-based if not enough data
        analysis_df['ML_Recommended_Action'] = analysis_df['Action_Label']
        analysis_df['Action_Confidence'] = 1.0
    
    return analysis_df

def generate_ml_recommendations(analysis_df):
    """
    Generate recommendations based on ML predictions
    """
    recommendations = []
    
    for _, row in analysis_df.iterrows():
        village = row['Village']
        mantri = row['Mantri_Name']
        mobile = row['Mantri_Mobile']
        taluka = row['Taluka']
        district = row['District']
        segment = row['Segment']
        action = row['ML_Recommended_Action']
        confidence = row['Action_Confidence']
        
        # Generate reason based on ML prediction
        if action == 'Send Marketing Team':
            reason = f"ML predicts marketing team needed (Confidence: {confidence:.2f}). Segment: {segment}"
            priority = 'High'
        elif action == 'Call Mantri for Follow-up':
            reason = f"ML predicts mantri follow-up needed (Confidence: {confidence:.2f}). Segment: {segment}"
            priority = 'High'
        elif action == 'Check on Mantri':
            reason = f"ML suggests checking on mantri (Confidence: {confidence:.2f}). Segment: {segment}"
            priority = 'Medium'
        elif action == 'Provide More Stock':
            reason = f"ML predicts stock increase needed (Confidence: {confidence:.2f}). Segment: {segment}"
            priority = 'Medium'
        else:
            reason = f"ML recommends regular follow-up (Confidence: {confidence:.2f}). Segment: {segment}"
            priority = 'Low'
        
        recommendations.append({
            'Village': village,
            'Taluka': taluka,
            'District': district,
            'Mantri': mantri,
            'Mobile': mobile,
            'Action': action,
            'Reason': reason,
            'Priority': priority,
            'Confidence': confidence,
            'Segment': segment,
            'Sales_Gap': row.get('Sales_Gap', 0)
        })
    
    return pd.DataFrame(recommendations)

def generate_ml_mantri_messages(recommendations):
    """
    Generate personalized messages based on ML recommendations
    """
    messages = []
    
    for _, row in recommendations.iterrows():
        if row['Action'] == 'Send Marketing Team':
            message = f"""
Namaste {row['Mantri']} Ji!

Our AI system has identified that your village {row['Village']} has high potential for growth. 
We're sending our marketing team to conduct demo sessions and help you reach more customers.

Based on our analysis:
- Segment: {row['Segment']}
- Confidence: {row['Confidence']*100:.1f}%

Please prepare for their visit and notify potential customers.

Dhanyavaad,
Calcium Supplement Team
"""
        elif row['Action'] == 'Call Mantri for Follow-up':
            message = f"""
Namaste {row['Mantri']} Ji!

Our AI analysis shows significant untapped potential in {row['Village']}. 
We recommend focusing on follow-up with these customers:

- Segment: {row['Segment']}
- Confidence: {row['Confidence']*100:.1f}%

A special commission offer is available for your next 10 customers.

Dhanyavaad,
Calcium Supplement Team
"""
        elif row['Action'] == 'Check on Mantri':
            message = f"""
Namaste {row['Mantri']} Ji!

Our system shows reduced activity in {row['Village']}. 
Is everything alright? Do you need any support from our team?

- Segment: {row['Segment']}
- Confidence: {row['Confidence']*100:.1f}%

Please let us know how we can help.

Dhanyavaad,
Calcium Supplement Team
"""
        elif row['Action'] == 'Provide More Stock':
            message = f"""
Namaste {row['Mantri']} Ji!

Great news! Our AI predicts increased demand in {row['Village']}. 
Would you like us to send additional stock?

- Segment: {row['Segment']}
- Confidence: {row['Confidence']*100:.1f}%
- Predicted Sales Gap: {row['Sales_Gap']:.1f}L

Please confirm your additional requirements.

Dhanyavaad,
Calcium Supplement Team
"""
        else:
            message = f"""
Namaste {row['Mantri']} Ji!

Our system shows steady performance in {row['Village']}. 
Keep up the good work!

- Segment: {row['Segment']}
- Confidence: {row['Confidence']*100:.1f}%

As always, let us know if you need any support.

Dhanyavaad,
Calcium Supplement Team
"""
        
        messages.append({
            'Mantri': row['Mantri'],
            'Mobile': row['Mobile'],
            'Village': row['Village'],
            'Action': row['Action'],
            'Message': message,
            'Priority': row['Priority'],
            'Confidence': row['Confidence']
        })
    
    return pd.DataFrame(messages)

# Example usage
def main():
    # Sample data
    data2=pd.read_excel("sampletesting.xlsx",sheet_name="Sheet1")
    data1=pd.read_excel("sampletesting.xlsx",sheet_name="Sheet2")
    
    # Generate ML-enhanced recommendations
    recommendations, analysis = enhanced_analyze_sales_data(data1, data2)
    
    print("ML-ENHANCED RECOMMENDATIONS:")
    print(recommendations.sort_values('Confidence', ascending=False).to_string(index=False))
    
    # Generate ML-based messages
    ml_messages = generate_ml_mantri_messages(recommendations)
    
    print("\nML-BASED MANTRI MESSAGES:")
    for _, msg in ml_messages.iterrows():
        print(f"\nTo: {msg['Mantri']} ({msg['Mobile']}) - {msg['Village']}")
        print(f"Action: {msg['Action']} (Confidence: {msg['Confidence']:.2f})")
        print(f"Message: {msg['Message']}")
    
    # Show village segments
    print("\nVILLAGE SEGMENTS:")
    segments = analysis[['Village', 'Segment', 'Conversion_Rate', 'Untapped_Potential', 'ML_Recommended_Action']]
    print(segments.to_string(index=False))
    
    return recommendations, ml_messages, analysis

if __name__ == "__main__":
    recommendations, ml_messages, analysis = main()

ML-ENHANCED RECOMMENDATIONS:
             Village    Taluka District                      Mantri       Mobile                    Action                                                                                           Reason Priority  Confidence                         Segment   Sales_Gap
               JILOD   ANKALAV    ANAND              AJAYBHAI PATEL   7984136988           Check on Mantri      ML suggests checking on mantri (Confidence: 1.00). Segment: High Potential - Low Engagement   Medium        1.00 High Potential - Low Engagement   -3.663333
                VADU     PADRA    PADRA              MANUBHAI PATEL   9909239416           Check on Mantri      ML suggests checking on mantri (Confidence: 1.00). Segment: High Potential - Low Engagement   Medium        1.00 High Potential - Low Engagement  -38.660000
              RANIYA    THASRA    KHEDA                MAHENDRABHAI   9727340474           Check on Mantri      ML suggests checking on mantri (Confidence: 1.00). S